In [1]:
import numpy as np
import cupy as cp
import tifffile as tiff
from skimage.morphology import skeletonize  # Import for skeletonization
from skan import Skeleton, summarize
from joblib import Parallel, delayed
import sys
from ltedt import local_thickness

In [2]:
binary_volume = tiff.imread("/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume.tif")
# binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume.tif")
# Get the size of the volume
print(f'Size of the volume is: {binary_volume.shape}')

Size of the volume is: (1035, 1706, 1510)


In [28]:
# Cut the volume in a smaller part
binary_volume_bottom_part = binary_volume[300:500,600:800,200:400]

# output_path = 'C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume_smaller.tif'
output_path = '/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume_smaller.tif'
tiff.imwrite(output_path, binary_volume_bottom_part)

In [32]:
smaller_skeleton = skeletonize(binary_volume_bottom_part)

# binarize the skeleton
smaller_skeleton = np.where(smaller_skeleton == True, 1, 0)

# output_path = 'C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume_smaller_skeleton.tif'
output_path = '/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume_smaller_skeleton.tif'
tiff.imwrite(output_path, smaller_skeleton)

In [ ]:
# Implement thickness with cuda
thickness = local_thickness(binary_volume_bottom_part, implementation="cupy")


In [ ]:
# Skeleton mask
thickness_map = thickness*smaller_skeleton


# Calcular el valor máximo en thickness_map, excluyendo los ceros
max_distance = np.max(thickness_map[thickness_map > 0])

print(f"Max distance in thickness map: {max_distance}")

Max distance in thickness map: 6


In [ ]:
skel = Skeleton(smaller_skeleton)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")


Number of branch type 0: 0
Number of branch type 1: 67
Number of branch type 2: 64
Number of branch type 3: 0


/tmp/ipykernel_83189/891031782.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [ ]:
print(branch_data.head())
print("Columnas disponibles:", branch_data.columns)

   skeleton-id  node-id-src  node-id-dst  branch-distance  branch-type  \
0            0            2           47        49.772952            1   
1            0           11           47        14.267220            1   
2            0           42           65         8.292529            1   
3            0           47           82        16.095647            2   
4            0           52           65         6.974691            1   

   mean-pixel-value  stdev-pixel-value  image-coord-src-0  image-coord-src-1  \
0               1.0                0.0                  0                180   
1               1.0                0.0                  4                143   
2               1.0                0.0                 13                130   
3               1.0                0.0                 14                143   
4               1.0                0.0                 16                128   

   image-coord-src-2  image-coord-dst-0  image-coord-dst-1  image-coord-ds

In [ ]:
# Parámetros de eliminación
x = 10  # Número mínimo de píxeles de la rama
thickness_threshold = 4  # Umbral de distancia en el thickness_map

# Inicializar una máscara para eliminar ramas que cumplen con los criterios
skeleton_cleaned = np.copy(smaller_skeleton)

# Iterar sobre cada rama en branch_data usando skeleton-id y node-id-src como identificador
for _, branch in branch_data.iterrows():
    branch_id = branch['skeleton-id']  # Puedes usar skeleton-id como identificador
    
    # Filtrar los píxeles de la rama actual, adaptando a tu estructura de datos
    # Aquí asumo que skel.branch_data podría ser similar a branch_data en su estructura
    branch_pixels = (skel.branch_data['skeleton-id'] == branch_id) & \
                    (skel.branch_data['node-id-src'] == branch['node-id-src'])

    # Verificar el tamaño de la rama
    num_pixels = np.sum(branch_pixels)

    # Calcular el valor máximo de espesor en thickness_map para esta rama
    max_thickness = thickness_map[branch_pixels].max()

    # Condición para eliminar la rama
    if num_pixels < x and max_thickness > thickness_threshold:
        # Eliminar la rama de la máscara esquelética
        skeleton_cleaned[branch_pixels] = 0

# Guardar el esqueleto limpio en un archivo
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume_smaller_skeleton_cleaned.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned)

print("El esqueleto ha sido limpiado y guardado en el archivo.")



AttributeError: 'Skeleton' object has no attribute 'branch_data'